In [ ]:
%%capture
#Get current directory
import os
curr_dir=os.getcwd()

# Download the Training file that contain model and framework
!wget -O face_landmark.zip https://www.dropbox.com/s/3l9qb414dzronaf/face_landmark.zip?dl=0
!unzip face_landmark.zip
!rm -rf face_landmark.zip
!pip install --upgrade --no-cache-dir gdown

In [ ]:
# Download COCO dataset label
%%capture
%cd {curr_dir}/face_landmark
!wget -O coco2017labels.zip https://www.dropbox.com/s/zd1ln3is8anb7rn/coco2017labels.zip?dl=0
!unzip -q coco2017labels.zip
!rm -rf coco2017labels.zip

In [ ]:
#Download COCO dataset (This will be used for representative dataset calibration on quantized tflite)
%cd coco/images
!f="val2017.zip" && curl http://images.cocodataset.org/zips/$f -o $f && unzip -q $f && rm $f  # 1G, 5k images

In [ ]:
%cd {curr_dir}/face_landmark/

#Renaming and moving dataset 
!mkdir datasets && mv ./coco/ ./datasets/

#Renaming path in dataset to match the latest path after moving dataset
!sed -i 's/.\/images/.\/datasets\/coco\/images/g' ./datasets/coco/val2017.txt

In [ ]:
#Convert model to .tflite
!python scripts/convert_tflite.py --model ./models/mediapipe_face_landmark_192x192x3/saved_model/ --input_shape 192x192x3 --int8_inputs --int8_operations --images ./datasets/coco/val2017.txt --random --num_images 100 --int8_outputs --min_value 0 --max_value 1

In [ ]:
#Perform inference on tflite model
!python scripts/perform_face_landmark_detection_inference.py --model ./tflite_model_192x192x3_int8_in_int8_opint8_out.tflite --image ./images/geoffrey-hinton.jpg --threshold 0.5


In [ ]:
#Rename tflite model following <architecture>_<application>.tflite. This will be used in TinyML Generator for accelerator and model generation
!mv {curr_dir}/face_landmark/tflite_model_192x192x3_int8_in_int8_opint8_out.tflite {curr_dir}/mediapipe_face_landmark.tflite